In [1]:
import requests
import time
from itertools import chain
from selenium import webdriver
import undetected_chromedriver as uc
from time import sleep
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

import json
import numpy as np
import tqdm.auto as tqdm
import contextlib
from bs4 import BeautifulSoup
import datetime
from mediumscrapper import ScrapeWebsite 


In [2]:
import contextlib
class FeedPage:
    def __init__(self):
        self.start_driver()

        title, content, rt, link, img_link = [], [], [], [], []
        for query in ["Machine+Learning", "Deep+Learning", "Python", "Programming", "Neural+Network", "CNN", "RNN", "GNN", "drug+discovery", "AI"]:
            t, c , r, l, i = self.QueryMaker(query = query)
            title.append(t)
            content.append(c)
            rt.append(r)
            link.append(l)
            img_link.append(i)
        myfilter = lambda x : x[:25]
        self.all_titles = list(chain.from_iterable(map(myfilter, title)))
        self.all_contents = list(chain.from_iterable(map(myfilter, content)))
        self.all_rt = list(chain.from_iterable(map(myfilter, rt)))
        self.all_links = list(chain.from_iterable(map(myfilter, link)))
        self.all_images = list(chain.from_iterable(map(myfilter, img_link)))
        self.scrapSite()
        self.makeHTML()
        self.driver.close()


    def start_driver(self):
        options = Options()
        #options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument("--no-sandbox")
        options.add_argument('--start-maximized')
        self.driver = uc.Chrome(exec_file = "/home/dibya/Dibyendu/Test/HTML/selenium/chromedriver/linux64/109.0.5414.74/chromedriver",options = options)
        
    def GiveR(self, url):
        r = requests.get(url, allow_redirects=False)
        if r.status_code != 301:
            self.driver.get(url)
            screen_height = self.driver.execute_script("return window.screen.height;")   # get the screen height of the web
            timeout = time.time() + 60*0.5
            scroll_pause_time = 0.5
            i = 1
            for _ in range(6):
                while True:
                    # scroll one screen height each time
                    self.driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
                    i += 1
                    time.sleep(scroll_pause_time)

                    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
                    scroll_height = self.driver.execute_script("return document.body.scrollHeight;")  
                    # Break the loop when the height we need to scroll to is larger than the total scroll height
                    if (screen_height) * i > scroll_height:
                        break
                r = self.driver.page_source
                bts = BeautifulSoup(r, 'html.parser').find_all("button")
                for i in range(len(bts)):
                    if bts[i].getText() == "Show more":
                        selector = bts[i]['class'][0]
                #btn = self.driver.find_elements(By.CLASS_NAME, selector)[0]
                #self.driver.execute_script("arguments[0].click();", WebDriverWait(self.driver, 30).until(EC.element_to_be_clickable(btn)))
                self.driver.execute_script("arguments[0].click();", WebDriverWait(self.driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Show more']"))))
            r = self.driver.page_source
            return r

    def QueryMaker(self, query):
        r = self.GiveR(f"https://medium.com/search?q={query}")
        soup = BeautifulSoup(r, 'html.parser')
        articles = soup.find_all('article')
        title_list, content_list, rt_list, link_list, img_list = [], [], [], [], []
        for article in articles:
            title, content, rt , link, img_link = self.getDetails(article)
            title_list.append(title)
            content_list.append(content)
            rt_list.append(rt)
            link_list.append(link)
            img_list.append(img_link)
        return title_list, content_list, rt_list, link_list, img_list

    def getDetails(self, article):
        all_a = article.find_all("a")
        title, content, rt, link, img_link = None, None, None, None, None  
        for a in all_a:
            with contextlib.suppress(Exception):
                #print(a.text)
                label = a.extract()['aria-label']
                #print(label)
                if label == "Post Preview Title":
                    title = a.find("h2").getText()
                    content = a.find("p").getText()
                elif label == "Post Preview Reading Time":
                    rt = a.getText()
                elif label == "Post Preview Image":
                    link = a.extract()['href']
                image = a.find('img')
                img_link = image.extract()['src']
                
        return title, content, rt, link, img_link

    def formatDate(self, date):
        return datetime.datetime.strptime(date,  '%b %d, %Y').strftime('%Y-%m-%d')
    
    def TagExtracter(self, Scrappable):
        All_As = Scrappable.Soup.find_all('a', {'rel':"noopener follow"})
        Checker = lambda x : 'tag' in x['href']
        tag_links = np.array(All_As, dtype = object)[list(map(Checker, All_As))]
        Tagger = lambda x : x.getText()
        return np.unique(list(map(Tagger, tag_links)))

    def formatTitle(self, title):
        remove_chars = ['[', ']', '/', '`', '"', "'", '…', '.', ',', '’', '‘', '\n', '“', '”', ':', '：', '，', "?", "!"]
        replace_with_space = [':']

        for char in remove_chars:
            title = title.replace(char, '')

        for char in replace_with_space:
            title = title.replace(char, ' ')

        title = '_'.join(title.split())

        return title
    def transform_data(self, author, title, content, date, tag, image, dir = "files"):

        # Replace single and double quotes
        author = author.replace("'", "").replace('"', '').replace('’', '').replace('‘', '')
        title = title.replace("'", "").replace('"', '').replace('’', '').replace('‘', '')
        content = content.replace("'", "").replace('"', '').replace('’', '').replace('‘', '').replace('…', '...')[:200]
        
        # Replace punctuation
        author = author.replace('\n','').replace('”', '').replace('“', '').replace(':', '').replace('：', '').replace('，', '')
        title = title.replace('\n','').replace('”', '').replace('“', '').replace(':', '').replace('：', '').replace('，', '')
        
        # Format Title & Date
        formated_title = self.formatTitle(title)
        formatted_date = self.formatDate(date)
        
        # Join tag list into string
        joined_tags = '_'.join(tag)
        
        # Return dictionary with transformed data
        return {
            "title": title,
            "content": content,
            "author": author,
            "image": image.replace("112", "640"),
            "tag": joined_tags,
            "dt": formatted_date,
            "file": f"{dir}/{formated_title}.php"
        }

    def getOne(self, image, title, content, date, file, author, tag):
        
        return f"""<div class="item features-image сol-12 col-md-6 col-lg-4" data-type="{tag}">
                        <div class="item-wrapper">
                            <div class="item-img">
                                <img src={image} alt="Image" loading="lazy" 
                                    title="">
                            </div>
                            <div class="item-content">
                                <h5 class="text-muted" style="font-size:15px; float:left;">
                                    <strong>{author}</strong>
                                    <em>&nbsp;{date}</em>
                                </h5>
                                <br>
                                <h6 class="item-subtitle mbr-fonts-style mt-1 display-7"><strong>{title}</strong>
                                </h6>
                                <p class="ff3">{content}...</p>
                            </div>
                            <div class="mbr-section-btn item-footer mt-2 text-center">
                                <a class="slide btn text-center" id="button" href="{file}" >&nbsp;</a></div>
                        </div>
                    </div>"""


    def GetDiv(self, detail_list ):
        forYou_text = """<section data-bs-version="5.1" class="content2 cid-sFAOw5Fdod" id="content2-e" style="padding-top: 0px; padding-bottom: 0px;">
                <div class="container">
                <div class="mbr-section-head">
                <br><br>
                    <h4 class="mbr-section-title mbr-fonts-style align-center mb-0 display-2"><strong>Free Articles from
                            Medium!</strong></h4>
                    <h5 class="mbr-section-subtitle mbr-fonts-style align-center mb-0 mt-2 display-5">Read the recent blog
                        about ML, DL and Data Science from Towards Data Science.</h5>
                </div><div class="row mt-4">"""

        for i in range(len(detail_list)):
            #print(i)
            #data = transform_data(all_authors[i], all_titles[i], all_contents[i], all_dates[i], all_tags[i], all_images[i], dir = "Feed")
            with contextlib.suppress(Exception):
                data = self.transform_data(self.all_authors[i], self.all_titles[i], self.all_contents[i], self.all_dates[i], self.all_tags[i], self.all_images[i], dir = "Feed")
                title = data['title']
                date = data['dt']
                image = data['image']
                author = data['author']
                content = data['content']
                file = data['file']
                tag = data['tag']
                #print(data)
                forYou_text += self.getOne(title=title, date=date, image=image, author=author, content=content, tag = tag, file = file)
        forYou_text += """</div></div></section>"""
        return forYou_text

    def scrapSite(self):
        Scrappable = ScrapeWebsite()
        author_list = []
        Address_list = []
        tag_list = []
        date_list = []
        for i in tqdm.trange(len(self.all_links)):
            with contextlib.suppress(Exception):
                if self.all_links[i] != None:
                    curl = f'https://medium.com{self.all_links[i]}'
                    Scrappable.DoScrap(curl, save_dir="./Feed/") 
                    Author, Address = Scrappable.GiveAuthor(Scrappable.Soup)
                    Date = Scrappable.Soup.find('p', {'class':'pw-published-date'}).getText()
                    tags = self.TagExtracter(Scrappable)
                    tag_list.append(tags)
                    date_list.append(Date)
                    Address_list.append(Address)
                    author_list.append(Author)
                else:
                    tag_list.append(None)
                    date_list.append(None)
                    Address_list.append(None)
                    author_list.append(None)
        self.all_authors = author_list
        self.all_address = Address_list
        self.all_dates = date_list
        self.all_tags = tag_list
    def makeHTML(self):
        head = open("Elemental/header.html").read()
        foot = open("Elemental/footer.html").read()
        with open("Feed2.html", "w") as f:
            f.write(head)
            f.write(self.GetDiv(self.all_address))
            f.write(foot)
            

In [5]:
F = FeedPage()

  0%|          | 0/250 [00:00<?, ?it/s]

WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=110.0.5481.100)
Stacktrace:
#0 0x5629c3139d93 <unknown>
#1 0x5629c2f0815d <unknown>
#2 0x5629c2ef23aa <unknown>
#3 0x5629c2ef0ee2 <unknown>
#4 0x5629c2ef1682 <unknown>
#5 0x5629c2efeb4f <unknown>
#6 0x5629c2eff7a2 <unknown>
#7 0x5629c2f0fdd1 <unknown>
#8 0x5629c2f80787 <unknown>
#9 0x5629c2f68353 <unknown>
#10 0x5629c2f37e40 <unknown>
#11 0x5629c2f39038 <unknown>
#12 0x5629c318d8be <unknown>
#13 0x5629c31918f0 <unknown>
#14 0x5629c3171f90 <unknown>
#15 0x5629c3192b7d <unknown>
#16 0x5629c3163578 <unknown>
#17 0x5629c31b7348 <unknown>
#18 0x5629c31b74d6 <unknown>
#19 0x5629c31d1341 <unknown>
#20 0x7f4800af5bb5 <unknown>


In [4]:
F.scrapSite()

NameError: name 'F' is not defined

In [2]:

links = np.load('links.npy', allow_pickle=True)

  0%|          | 0/87 [00:00<?, ?it/s]